In [1]:
#!/usr/bin/env Rscript

LABEL_COLS <- c("click", "basket", "order", "revenue")

library(data.table)
library(stringr)

IN <- list(
  items = "../data/raw/final_unif.csv",
  train = "../data/raw/train.csv",
  test  = "../data/raw/class.csv")

OUT <- list(
  train = "../data/raw/nolabel_feat_train.csv",
  test  = "../data/raw/nolabel_feat_test.csv")

BREAKS <- c(21, 14, 14, 14, 14, 15)

#读取数据
items <- read.table(IN$items, sep = ",", header=TRUE)
train <- read.table(IN$train, sep = "|", header=TRUE)
test <- read.table(IN$test, sep = "|", header=TRUE)

Warning message:
"package 'data.table' was built under R version 3.5.1"Warning message:
"package 'stringr' was built under R version 3.5.1"

In [2]:
#------------------------2. content特征-----------------------------
#处理item content的内容，把里面的a x b x c 中用“x”连接的部分分开，变成数字。
split_units = str_split_fixed(items$content, "x", 3)
dims = dim(split_units)
split_units = str_replace_all(split_units, "[[:space:][:alpha:]]", "")
split_units[split_units == ""] = "1"
split_units = as.numeric(split_units)
dim(split_units) = dims

#特征2.1
items$content_part1 = split_units[, 1]
#特征2.2
items$content_part2 = split_units[, 2]
#特征2.3
items$content_part3 = split_units[, 3]
#特征2.4
items$total_units = items$content_part1 * items$content_part2 * items$content_part3

items <- subset(items, select = -content_part1 )
items <- subset(items, select = -content_part2 )
items <- subset(items, select = -content_part3 )
items <- subset(items, select = -content_part4 )

ERROR: Error in eval(substitute(select), nl, parent.frame()): 找不到对象'content_part4'


In [4]:
items <- subset(items, select = -pid1 )

In [7]:
# ==================== ASSEMBLE LINES ====================
# 对train进行处理
labels <- train[, c("lineID", LABEL_COLS)]
train <- train[c(1,2,3,4,5,6,10)]
final_train_day <- max(train$day)

#把train 和 test 结合起来
df <- rbind(train, test)

#把items和df文件结合起来
df <- merge(df, items, by = "pid")
df = df[order(df$lineID), ]
rm(train, test, items)
df = data.table(df)
setkey(df, lineID)

In [8]:
#------------------------5. 填补竞争者价格---------------------------------
df$day_mod_7 = df$day %% 7

df$competitorPrice_imputed = df$competitorPrice

#特征5.2：items 中重复的pid
df$deduplicated_pid = duplicated(df$pid)

#用相同产品的平均值来填补缺失价格，以及其他类似的价格来填补，上述不能填补的用价格来填补
#特征5.3:填补后竞争者价格
df[, competitorPrice_imputed :=
  ifelse(is.na(competitorPrice_imputed)
    , mean(competitorPrice, na.rm = TRUE)
    , competitorPrice_imputed)
  , by = pid]
df[, competitorPrice_imputed :=
  ifelse(is.na(competitorPrice_imputed)
    , mean(competitorPrice, na.rm = TRUE)
    , competitorPrice_imputed)
  , by = deduplicated_pid]
df[, competitorPrice_imputed :=
  ifelse(is.na(competitorPrice_imputed)
    , mean(competitorPrice, na.rm = TRUE)
    , competitorPrice_imputed)
  , by = .(group, content, unit, day_mod_7, salesIndex, adFlag)]
df[, competitorPrice_imputed :=
  ifelse(is.na(competitorPrice_imputed)
    , mean(competitorPrice, na.rm = TRUE)
    , competitorPrice_imputed)
  , by = .(group, content, unit)]
df[is.na(competitorPrice_imputed), competitorPrice_imputed := price]

#删除了原本的竞争者价格，变成了经过填补之后的竞争者价格
df <- subset(df, select = -competitorPrice )
df <- subset(df, select = -day_mod_7 )

In [9]:
#------------------------6. 价格特征---------------------------------
#平均每单位商品的价格
#特征6.1：平均每个单位商品的参考价格
#特征6.2：平均每个单位商品的实际销售价格
#特征6.3：平均每个单位商品的竞争者价格
df$rrp_per_unit = df$rrp /df$total_units
df$price_per_unit = df$price / df$total_units
df$competitorPrice_per_unit = df$competitorPrice_imputed / df$total_units

#商品的价格差和折扣差
#特征6.4：实际销售价格和竞争者价格的差价
#特征6.5：自身折扣
#特征6.6：竞争者的折扣
#特征6.6：自身和竞争者的折扣差
df$price_diff = df$price - df$competitorPrice_imputed
df$price_discount = (df$price - df$rrp) / df$rrp
df$competitorPrice_discount = (df$competitorPrice_imputed - df$rrp) / df$rrp
df$price_discount_diff = df$price_discount - df$competitorPrice_discount

#是否处于优势地位？
#特征6.8：是否比竞争者更低价
#特征6.9：是否比竞争者更低折扣
#特征6.10：是否比竞争者更低折扣
df$is_lower_price = as.integer(df$price_diff < 0)
df$is_discount = as.integer(df$price_discount < 0)
df$isgreater_discount = as.integer(df$price_discount_diff < 0)

#特征6.11、12、13、14、15、16，本商品折扣在所有同类商品中折扣的多少
df[, `:=`(
    price_discount_min = min(price_discount)
    , price_discount_p25 = quantile(price_discount, probs = 0.25)
    , price_discount_med = median(price_discount)
    , price_discount_p75 = quantile(price_discount, probs = 0.75)
    , price_discount_max = max(price_discount) 
  ), by = "deduplicated_pid"]

df <- subset(df, select = -deduplicated_pid )

In [67]:
# #------------------------7. 计数特征---------------------------------
# #按照时间周期和不同变量进行分类后统计商品数量
# #特征7.1、2、3、4、5、6、7、8
# df[, content_d7cnt := log(.N), by = .(content, day_mod_7)]
# df[, group_d7cnt := log(.N), by = .(group, day_mod_7)]
# df[, manufacturer_d7cnt := log(.N), by =.(manufacturer, day_mod_7)]
# df[, unit_d7cnt := log(.N), by = .(unit, day_mod_7)]
# df[, pharmForm_d7cnt := log(.N), by = .(pharmForm, day_mod_7)]
# df[, category_d7cnt := log(.N), by = .(category, day_mod_7)]
# df[, campaignIndex_d7cnt := log(.N), by = .(campaignIndex, day_mod_7)]
# df[, salesIndex_d7cnt := log(.N), by = .(salesIndex, day_mod_7)]

# #按照更加复杂的方式进行统计的变量
# #特征7.9、10、11、12
# df[, inter_gcucd7_cnt := log(.N),
#   by = .(group, content, unit, campaignIndex, day_mod_7)]
# df[, inter_gcucd10_cnt := log(.N),
#   by = .(group, content, unit, adFlag, day_mod_10)]
# df[, inter_gcucd30_cnt := log(.N),
#   by = .(group, content, unit, availability, day_mod_30)]
# df[, inter_gcuca_cnt := log(.N),
#   by = .(group, content, unit, campaignIndex, availability, adFlag)]

In [68]:
#------------------------8. 价格趋势特征---------------------------------
# shift_mean = function(x, n, ...) Reduce("+", shift(x, 1:n, ...)) / n

# setkey(df, lineID, pid)

# #特征8.1：前一天价格
# #特征8.2：前五天价格均值
# #特征8.3：前五天价格均值最小值
# #特征8.4：前五天价格均值最大值
# #特征8.5：未来一天价格
# #特征8.6：未来五天价格均值
# #特征8.7：未来五天价格均值最小值
# #特征8.8：未来五天价格均值最大值
# df[, `:=`(
#     prev_price = shift(price, 1, median(price))
#     , prev5_price_avg = shift_mean(price, 5, price[[1]])
#     , prev5_price_min = do.call(pmin, shift(price, 1:5, price[[1]]))
#     , prev5_price_max = do.call(pmax, shift(price, 1:5, price[[1]]))

#     , next_price = shift(price, 1, median(price), type = "lead")
#     , next5_price_avg = shift_mean(price, 5, median(price), type = "lead")
#     , next5_price_min = do.call(pmin, shift(price, 1:5, price[[1]]))
#     , next5_price_max = do.call(pmax, shift(price, 1:5, price[[1]]))
#   ), by = pid]
# #特征8.9：当前降价比例
# #特征8.10：过去五天价格差
# #特征8.11：当前价格是否高于过去五天价格
# #特征8.12：未来可能的降价比例
# #特征8.13：未来五天价格差
# #特征8.14：当前价格是否可能高于未来五天价格
# df[, `:=`(
#     prev_price_pct_chg = (price - prev_price) / prev_price
#     , prev5_price_diff = prev5_price_max - prev5_price_min
#     , price_gt_prev5 = as.integer(price > prev5_price_avg)

#     , next_price_pct_chg = (next_price - price) / price
#     , next5_price_diff = next5_price_max - next5_price_min
#     , price_lt_next5 = as.integer(price < next5_price_avg)
#   )]

In [22]:
summary(df)

      pid            lineID             day             adFlag      
 Min.   :    1   Min.   :      1   Min.   :  1.00   Min.   :0.0000  
 1st Qu.: 4236   1st Qu.: 495847   1st Qu.: 39.00   1st Qu.:0.0000  
 Median :10005   Median : 991693   Median : 69.00   Median :0.0000  
 Mean   :10392   Mean   :1142178   Mean   : 67.67   Mean   :0.3227  
 3rd Qu.:16091   3rd Qu.:1764311   3rd Qu.: 98.00   3rd Qu.:1.0000  
 Max.   :22035   Max.   :2756003   Max.   :123.00   Max.   :1.0000  
                                                                    
  availability      price         manufacturer         group        
 Min.   :1.00   Min.   :  0.01   Min.   :   1.0   22OI3   : 224606  
 1st Qu.:1.00   1st Qu.:  5.95   1st Qu.:  56.0   2FOI    : 214444  
 Median :1.00   Median :  9.85   Median : 105.0   21OKG   : 155037  
 Mean   :1.11   Mean   : 13.73   Mean   : 173.7   18OI0EOI: 138827  
 3rd Qu.:1.00   3rd Qu.: 16.45   3rd Qu.: 220.0   21OK3   : 104867  
 Max.   :4.00   Max.   :384.71   M

[1] 0

In [25]:
# ==================== PARTITION LINES ====================
train <- df[day <= final_train_day]
test <- df[day > final_train_day]

train = data.table(train)
test = data.table(test)

train = train[order(train$lineID), ]
test = test[order(test$lineID), ]

train <- merge(train, labels, by = "lineID")

In [26]:
#------------------------9. train中特有的特征---------------------------------
#特征9.1：销售数量
train[, `:=`(
    order_qty = as.integer(revenue / price)
  )]

In [27]:
# ==================== WRITE TO DISK ====================
write.table(train, "../data/raw/train_wfb.csv")
write.table(test, "../data/raw/test_wfb.csv")